In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from urllib.parse import urlparse, parse_qs
import os
from ast import literal_eval

In [5]:
directory = '/project/ll_774_951/uk_ru/twitter/data/2022-02'
save_directory1 = '/scratch1/jinyiy/link_count'
save_directory2 = '/scratch1/jinyiy/link_fulldata'
filenames =['ukraine_russia-2022-02-22-04.csv', 'ukraine_russia-2022-02-22-05.csv']

In [10]:
for filename in filenames:
    f = os.path.join(directory, filename)
    print(f, 'processing...')
    df = pd.read_csv(f, error_bad_lines = False, engine = 'python', warn_bad_lines = False)
    print('df created')
    

/project/ll_774_951/uk_ru/twitter/data/2022-02/ukraine_russia-2022-02-22-04.csv processing...
df created
/project/ll_774_951/uk_ru/twitter/data/2022-02/ukraine_russia-2022-02-22-05.csv processing...
df created


In [ ]:
for filename in filenames:
    try: 
        f = os.path.join(directory, filename)
        print(f, 'processing...')

        df = pd.read_pickle(f, compression='infer')
        df = pd.DataFrame(df)

        df['expandedLink'] = df['expandedLinks'].apply(pd.Series)[0].apply(pd.Series).iloc[:,1]
        df['domain'] = df['expandedLink'].apply(lambda x: urlparse(str(x)).netloc)

        link_df = pd.DataFrame(df['expandedLink'].groupby(df['type']).value_counts())
        link_df = link_df.add_suffix('_Count').reset_index()
        link_df['domain'] = link_df['expandedLink'].apply(lambda x: urlparse(str(x)).netloc)
        link_df = link_df.sort_values('expandedLink_Count',ascending=False).reset_index()

        link_df_50 = link_df.sort_values('expandedLink_Count',ascending=False)[0:50].reset_index()
        link_df_50_fulldata = pd.DataFrame()
        for link in link_df_50['expandedLink']:
            link_df_50_fulldata = link_df_50_fulldata.append(df.loc[df['expandedLink'] == link])

        s1 = os.path.join(save_directory1, filename + ".csv")
        link_df.to_csv(s1, sep=',')
        print(filename, 'completed1')
        
        s2 = os.path.join(save_directory2, filename + ".csv")
        link_df_50_fulldata.to_csv(s2, sep=',')
        print(filename, 'completed2')
    
    except:
        continue

In [ ]:
# Get a dataframe of all the link_count files, rank the links and save to csv
total_link_count = pd.DataFrame()
for file in sorted(os.listdir(save_directory1)):
    f = os.path.join(save_directory1, file)
    df = pd.read_csv(f)
    total_link_count = total_link_count.append(df)
    print(file, 'completed')
    

In [ ]:
total_link_count.head()

In [ ]:
total_link_count.to_csv('/scratch1/jinyiy/total_link_raw.csv', sep=',')

In [ ]:
total_link_count = pd.read_csv('/scratch1/jinyiy/total_link_raw.csv', sep=',')

In [ ]:
total_link_count = total_link_count.groupby('expandedLink').sum().reset_index()

In [ ]:
total_link_count['domain'] = total_link_count['expandedLink'].apply(lambda x: urlparse(str(x)).netloc)

In [ ]:
total_link_count = total_link_count.sort_values('expandedLink_Count', ascending=False).reset_index()

In [ ]:
total_link_count = total_link_count.drop('level_0', axis=1)
total_link_count = total_link_count.drop('index', axis=1)
total_link_count

In [ ]:
total_link_count.to_csv('/scratch1/jinyiy/total_link_count.csv', sep=',')

In [ ]:
total_link_count[0:1000].to_csv('/scratch1/jinyiy/top1000_link.csv', sep=',')

In [ ]:
total_link_count = pd.read_csv('/scratch1/jinyiy/total_link_count.csv', sep=',')

In [ ]:
total_domain_count = total_link_count.groupby('domain').sum().reset_index()

In [ ]:
total_domain_count = total_domain_count.sort_values('expandedLink_Count', ascending=False).reset_index()

In [ ]:
total_domain_count = total_domain_count.drop('index', axis=1)
total_domain_count = total_domain_count.drop('Unnamed: 0', axis=1)
total_domain_count

In [ ]:
total_domain_count.to_csv('/scratch1/jinyiy/total_domain_count.csv', sep=',')

In [ ]:
total_domain_count[0:1000].to_csv('/scratch1/jinyiy/top1000_domain.csv', sep=',')

In [ ]:
# Get a dataframe of all the link_fulldata files and save to csv
total_link_data_1 = pd.DataFrame()
for file in filenames:
    f = os.path.join(save_directory2, file)
    df = pd.read_csv(f)
    total_link_data_1 = total_link_data_1.append(df)
    print(file, 'completed')

In [ ]:
total_link_data_1.to_csv('/scratch1/jinyiy/total_link_data1.csv', sep=',')

In [ ]:
# Get posts regarding specific URLs
doc_df = pd.read_csv('/scratch1/jinyiy/total_link_data1.csv')

In [ ]:
doc_df2 = pd.read_csv('/scratch1/jinyiy/total_link_data2.csv')

In [ ]:
doc_df3 = pd.read_csv('/scratch1/jinyiy/total_link_data3.csv')

In [ ]:
urls = []
save_directory = '/scratch1/jinyiy/URL_top50'

for i in range(len(urls)):
    df = pd.DataFrame()
    print('Searching', urls[i], '...')
    for doc in [doc_df, doc_df2, doc_df3]:
        df = df.append(doc.loc[doc['expandedLink'] == urls[i]])
    print('Search complete', urls[i])
    s = os.path.join(save_directory, "url_" + str(i+1) + ".csv")
    df.to_csv(s, sep=',')
    print(s, 'saved!')
        
    

In [ ]:
doc_df2.loc[doc_df2['expandedLink'] == 'https://instagram.com/iloveanimals.group?utm_medium=copy_link']

In [2]:
# plotting by date
import plotly.express as px
import plotly.graph_objs as go

In [3]:
df = pd.read_csv('/scratch1/jinyiy/URL_top50/url_1.csv', sep=',')
df['day'] = df['date'].apply(lambda x: x.split()[0])
by_date = pd.Series(df['day']).value_counts().sort_index()
by_date.index = pd.DatetimeIndex(by_date.index)
df_date = by_date.rename_axis('date').reset_index(name='counts')
df_date

,date,counts
0,2022-03-06,774
1,2022-03-07,628
2,2022-03-08,881
3,2022-03-09,878
4,2022-03-10,1056
...,...,...
72,2022-05-17,638
73,2022-05-18,735
74,2022-05-19,748
75,2022-05-20,679


In [4]:
fig = go.Figure(data=go.Scatter(x=df_date['date'].astype(dtype=str), 
                                y=df_date['counts'],
                                marker_color='black', text="counts"))
fig.update_layout({"title": 'Posts with link https://t.me/visnovosti Day by Day',
                   "xaxis": {"title":"Time"},
                   "yaxis": {"title":"Total posts"},
                   "showlegend": False})
fig.write_image('/scratch1/jinyiy/URL_top50/url_1.png') 

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
! pip install -U kaleido